# Import Libraries

In [2]:
#For Scraping
import requests
from bs4 import BeautifulSoup as bs 
from bs4 import SoupStrainer
import httplib2
import pandas as pd
import pandas_datareader as web
from lxml import html

#Exporting to excel
from openpyxl import load_workbook

import matplotlib.pyplot as plt

#Formatting
from datetime import date, timedelta, datetime
import numpy as np 


# Create function for scraping information from website

In [7]:
def find_table(year, day):
	url = "https://www.worldfootball.net/schedule/bundesliga-" + str(year) + "-spieltag/" +str(day)+ "/"

	page = requests.get(url)

	tabelle = {}
	bs_t = bs(page.text, 'lxml')

	list_all_keys = bs_t.findAll('table', class_="standard_tabelle")

	for key in list_all_keys:

		text = key.findAll("tr")

		for k in text:
			heimsieg = 0
			unentschieden = 0
			auswaertssieg = 0

			t = k.findAll("td")

			#Len indicator so only the table is scraped
			if len(t) == 10:
				tabellenplatz = t[0].text
                # If teams share a table spot, one of them does not have a number. This snipped
                # then gives the team the last used table spot
				if tabellenplatz == "\xa0":
					tabellenplatz = var_value
				elif not tabellenplatz =="\xa0":
					var_value = tabellenplatz
                #scrape other information from table (Name, Wins, Draws, Losses, Goal Difference, Points)
				name = t[2].text.strip()
				wins = t[4].text.strip()
				draws = t[5].text.strip()
				loss = t[6].text.strip()
				goal_diff = int(t[8].text.strip())
				punkte =  int(t[9].text.strip())

				tabelle[name] = (tabellenplatz, wins, draws, loss, goal_diff, punkte)


	return tabelle

# Start Scraping

In [8]:
#Initialize DataFrame
columns = ["Jahr", "Spieltag", "Heimteam", "Auswaertsteam", "Tore Heim", "Tore Ausw", "Heimsieg", "Unentschieden", "Auswaertssieg", "Tabelle Heim", "Tabelle Auswaerts"]
df1 = pd.DataFrame(columns = columns)

#Itarate through years
s = 0
y = 2000
for i in range(2020-19, 2021, 1):
	daten = str(y) + "-" + str(i)
	y = i
	s += 1
	status = (s/len(range(2020-19, 2021, 1))) * 100
	print("Download status=", status, "%")
    #Iterate through Game Days in a season
	for i in range(1,35):
        #Get Table Data from function
		tabelle = find_table(daten, i)

		url = "https://www.worldfootball.net/schedule/bundesliga-" + str(daten) + "-spieltag/" + str(i) + "/"

		page = requests.get(url)

		bs_t = bs(page.text, 'lxml')

		list_all_keys = bs_t.findAll('table', class_="standard_tabelle")

		for key in list_all_keys:
			#Table spot
			text = key.findAll("tr")

			for k in text:
				heimsieg = 0
				unentschieden = 0
				auswaertssieg = 0

				t = k.findAll("td")

				#Only scrape Results
				if len(t) == 7:
                    #Result
					ergebnis = t[5].text.split("(")[0].split(":")
                    #Home Team goals
					heim_tore = int(ergebnis[0])
                    #away team goals
					ausw_tore = int(ergebnis[1])
                    #home team
					heimteam = t[2].text.strip()
                    #away team
					auswteam = t[4].text.strip()

					#One hot encoding for winner
					if heim_tore < ausw_tore:
						auswaertssieg = 1
					elif heim_tore == ausw_tore:
						unentschieden = 1
					elif heim_tore > ausw_tore:
						heimsieg = 1
                        
					#fill DataFrame
					df1 = df1.append({"Jahr": daten, "Spieltag": i, "Heimteam": heimteam, "Auswaertsteam": auswteam, "Tore Heim":heim_tore, "Tore Ausw":ausw_tore, "Heimsieg": heimsieg, "Unentschieden": unentschieden,
						 "Auswaertssieg":auswaertssieg, "Tabelle Heim": tabelle[heimteam][0], "Tabelle Auswaerts": tabelle[auswteam][0],"WinsHome": tabelle[heimteam][1], "WinsAway": tabelle[auswteam][1],
						 "DrawsHome": tabelle[heimteam][2], "DrawsAway": tabelle[auswteam][2],
						 "LossHome": tabelle[heimteam][3], "LossAway": tabelle[auswteam][3],
						 "GoalDiffHome": tabelle[heimteam][4], "GoalDiffAway": tabelle[auswteam][4],
						 "PunkteHome": tabelle[heimteam][5], "PunkteAway": tabelle[auswteam][5]}, ignore_index=True)  

df1.head()

Download status= 5.0 %
Download status= 10.0 %
Download status= 15.0 %
Download status= 20.0 %
Download status= 25.0 %
Download status= 30.0 %
Download status= 35.0 %
Download status= 40.0 %
Download status= 45.0 %
Download status= 50.0 %
Download status= 55.00000000000001 %
Download status= 60.0 %
Download status= 65.0 %
Download status= 70.0 %
Download status= 75.0 %
Download status= 80.0 %
Download status= 85.0 %
Download status= 90.0 %
Download status= 95.0 %
Download status= 100.0 %


,Jahr,Spieltag,Heimteam,Auswaertsteam,Tore Heim,Tore Ausw,Heimsieg,Unentschieden,Auswaertssieg,Tabelle Heim,...,DrawsAway,DrawsHome,GoalDiffAway,GoalDiffHome,LossAway,LossHome,PunkteAway,PunkteHome,WinsAway,WinsHome
0,2000-2001,1,Borussia Dortmund,Hansa Rostock,1,0,1,0,0,7,...,0,0,-1.0,1.0,1,0,0.0,3.0,0,1
1,2000-2001,1,1. FC Kaiserslautern,VfL Bochum,0,1,0,0,1,12,...,0,0,1.0,-1.0,0,1,3.0,0.0,1,0
2,2000-2001,1,Bayer Leverkusen,VfL Wolfsburg,2,0,1,0,0,5,...,0,0,-2.0,2.0,1,0,0.0,3.0,0,1
3,2000-2001,1,SC Freiburg,VfB Stuttgart,4,0,1,0,0,1,...,0,0,-4.0,4.0,1,0,0.0,3.0,0,1
4,2000-2001,1,Werder Bremen,Energie Cottbus,3,1,1,0,0,4,...,0,0,-2.0,2.0,1,0,0.0,3.0,0,1


In [ ]:
#Export to Excel for later usage
df1.to_excel("Bundesliga_Ergebnisse.xlsx")